In [1]:
# Hyperparameters:
batch_size = 128

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torchvision.models as models
import kagglehub
import os
import pandas as pd
import numpy as np
from datetime import timedelta
import math
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error, root_mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error

In [3]:
# Replace nan with 0
def preprocess_tensor(tensor):
    return torch.nan_to_num(tensor, nan=0.0)  # Replace NaN with 0.0

# Load and process data
test_set = torch.load("./Datasets/test_set.pt", weights_only=False)
test_data = [(preprocess_tensor(x), y) for x, y in test_set]

# Create DataLoaders
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [4]:
# Define Linear Regression Model
class SimpleLinearRegression(nn.Module):
    def __init__(self, input_size):
        super(SimpleLinearRegression, self).__init__()
        self.linear = nn.Linear(input_size, 1)  # One output node for regression

    def forward(self, x):
        return self.linear(x)

In [5]:
# Initialize the model
input_size = test_set[0][0].shape[0]
model = SimpleLinearRegression(input_size)

# Load the model state dictionary
model.load_state_dict(torch.load('./Models/lin_reg.pth'))
model.eval()  # Set the model to evaluation mode

C:\Users\darth\AppData\Local\Temp\ipykernel_22880\734795541.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./Models/lin_reg.pth'))


SimpleLinearRegression(
  (linear): Linear(in_features=196, out_features=1, bias=True)
)

In [6]:
# Lists to store actual and predicted values
y_true = []
y_pred = []

# Disable gradient calculations during evaluation
with torch.no_grad():
    for inputs, targets in test_loader:
        # Forward pass: get predictions
        outputs = model(inputs)
        
        # Append actual and predicted values to the lists
        y_true.extend(targets.numpy())
        y_pred.extend(outputs.numpy())

# Calculate evaluation metrics
mse = mean_squared_error(y_true, y_pred)
rmse = root_mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)
mape = mean_absolute_percentage_error(y_true, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"Root MSE: {rmse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"R² Score: {r2:.4f}")
print(f"Mean Absolute Percentage Error: {mape:.4f}")

Mean Squared Error: 310.3066
Root MSE: 17.6155
Mean Absolute Error: 14.3498
R² Score: 0.8746
Mean Absolute Percentage Error: 0.2749
